In [2]:
import numpy as np
import cv2
import time

MIN_MATCH_COUNT = 20  #мінімальна кількість спільних ознак яка потрібна для розпізнавання об'єкта
QUERY_IMAGE = "main.jpg" #Об'єкт, який потрібно розпізнати

In [8]:
#Код, який рахує метрики

printImages = True

ametric1 = 0
ametric2 = 0
ametric3 = 0
ndetected = 0
print("image\t1 metric\t2 metric\t3 metric\tgood matches")

img1 = cv2.imread(QUERY_IMAGE, cv2.IMREAD_GRAYSCALE)
#img1 = cv2.resize(img1, (0,0), fx=0.1, fy=0.1) 

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)

for i in range(120):
    img2 = cv2.imread(f"{1+i}.jpg", cv2.IMREAD_GRAYSCALE)
    #img2 = cv2.resize(img2, (0,0), fx=0.2, fy=0.2)
    
    start_time = time.time()
    kp2, des2 = surf.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    
    #good = bf.match(des1, des2)  
    #good = sorted(matches, key = lambda x:x.distance)
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)
    
    if (len(good)>=MIN_MATCH_COUNT):
        ndetected += 1
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        
        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)

        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        
        corect = 0
        sum = 0
        for j in range(len(mask)):
            if mask[j][0] == 1:
                corect+=1
                sum += good[j].distance
        metric1 = round(corect / len(mask), 4)
        metric2 = round(sum / corect, 4)
        metric3 = round((time.time() - start_time)*10**9/(img.shape[0] * img.shape[1]), 3)
        print(f"{1+i}:\t{metric1}\t\t{metric2}\t\t{metric3}\t\t{len(good)}")
        ametric1 += metric1
        ametric2 += metric2
        ametric3 += metric3

    else:
        print (f"{1+i}:\tObject has not detected. Not enough matches are found - {len(good)}/{MIN_MATCH_COUNT}")
        matchesMask = None
    start_time = time.time()
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    
    if(printImages):
        img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

        cv2.imshow("Matching result", img3)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
print(f"Average:\n1 metric: {ametric1/ndetected}\n2 metric: {ametric2/ndetected}\n3 metric: {ametric3/ndetected}")

image	1 metric	2 metric	3 metric	good matches
1:	0.7021		0.2128		96.34		47
2:	0.6897		0.2109		123.36		29
3:	0.7358		0.1877		119.533		53
4:	0.5405		0.1727		119.505		37
5:	0.625		0.2168		111.792		40
6:	0.3182		0.2443		123.387		22
7:	0.6452		0.2072		331.546		31
8:	0.7576		0.1993		312.258		33
9:	0.7143		0.1661		329.605		49
10:	0.6341		0.1953		221.661		41
11:	0.7297		0.1748		231.3		74
12:	0.7581		0.1782		208.171		62
13:	0.44		0.2104		385.504		25
14:	0.5909		0.207		225.514		44
15:	0.8372		0.1771		510.768		43
16:	0.6923		0.2157		302.616		26
17:	Object has not detected. Not enough matches are found - 16/20
18:	0.7372		0.1616		109.874		156
19:	0.3333		0.1956		146.472		39
20:	0.2903		0.1882		171.534		31
21:	0.3333		0.2281		136.848		21
22:	0.5429		0.1819		144.558		35
23:	0.5		0.2089		140.705		30
24:	0.6111		0.1951		136.83		54
25:	0.6739		0.1907		117.575		46
26:	0.7381		0.2161		136.85		42
27:	0.5909		0.1978		138.803		22
28:	0.6		0.1834		146.477		55
29:	0.5172		0.2177		173.411		29
30:	0.5		0.2258		

In [37]:
#Код для обробки конкретного зображення

Image = "61.jpg"

img1 = cv2.imread(QUERY_IMAGE, cv2.IMREAD_GRAYSCALE)
#img1 = cv2.resize(img1, (0,0), fx=0.1, fy=0.1) 

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)

img2 = cv2.imread(Image, cv2.IMREAD_GRAYSCALE)
#img2 = cv2.resize(img2, (0,0), fx=0.2, fy=0.2)
kp2, des2 = surf.detectAndCompute(img2, None)
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
"""        
sum = 0
for m in good:
    sum += m.distance
print(f"{sum / len(good)}")
"""
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print ("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None


draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

#plt.imshow(img3, 'gray'),plt.show()
    
#matching_result = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None, flags=2)
#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Matching result", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

Not enough matches are found - 11/20
26
11


In [7]:
for i in range(2):
    img = cv2.imread(f"{i+62}.jpg")
    #if i > 102 or i > 5 and i < 11:
    img = cv2.resize(img, (0,0), fx=0.2, fy=0.2)
    #else:
        #img = cv2.resize(img, (0,0), fx=0.1, fy=0.1)
    cv2.imwrite(f"{i+62}.jpg", img)